In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

# Set the base path to the current working directory
base_path = os.getcwd()

# Load the saved models with relative paths
ethnicity_model_path = os.path.join(base_path,'..', 'saved models', 'ethinicity_detector_task5.model')
emotion_model_path = os.path.join(base_path,'..', 'saved models', 'Human_expression_task_5.model')
age_model_path = os.path.join(base_path,'..', 'saved models', 'age_detector_task5.model')
dress_color_model_path = os.path.join(base_path,'..', 'saved models', 'dress_color_detector_task5.model')

ethnicity_model = load_model(ethnicity_model_path)
emotion_model = load_model(emotion_model_path)
age_model = load_model(age_model_path)
dress_color_model = load_model(dress_color_model_path)

# Function to preprocess the image
def preprocess_image(image_path, target_size, grayscale=True):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found or unable to load: {image_path}")
    image = cv2.resize(image, target_size)  # Resize to target size
    if grayscale:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        image = np.expand_dims(image, axis=-1)  # Add channel dimension
    image = image / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Function to map ethnicity
def map_ethnicity(ethnicity):
    if ethnicity == 0:
        return 'White'
    elif ethnicity == 1:
        return 'Black'
    elif ethnicity == 2:
        return 'Asian'
    elif ethnicity == 3:
        return 'Indian'
    elif ethnicity == 4:
        return 'Others'
    return 'Others'

# Function to map emotion
def map_emotion(emotion):
    emotion_labels = ['surprise', 'sad', 'neutral', 'happy', 'fear', 'disgust', 'angry']  # Updated labels
    if emotion < len(emotion_labels):
        return emotion_labels[emotion]
    return 'Unknown'

# Function to predict nationality and emotion
def predict_nationality_and_emotion(image):
    ethnicity_pred = ethnicity_model.predict(image)
    emotion_pred = emotion_model.predict(image)
    nationality = np.argmax(ethnicity_pred, axis=1)[0]
    emotion = np.argmax(emotion_pred, axis=1)[0]
    return map_ethnicity(nationality), map_emotion(emotion)

# Function to predict age
def predict_age(image):
    age_pred = age_model.predict(image)
    age_bins = [0, 20, 40, 60, 80, 100]  # Defined age bins
    age = np.argmax(age_pred, axis=1)[0]
    return age_bins[age], age_bins[age + 1]  # Return age range

# Function to predict dress color
def predict_dress_color(image):
    dress_color_pred = dress_color_model.predict(image)
    dress_color = np.argmax(dress_color_pred, axis=1)[0]
    dress_color_labels = ['Red', 'Green', 'Blue', 'Black', 'White']  # Example labels
    if dress_color < len(dress_color_labels):
        return dress_color_labels[dress_color]
    return 'Unknown'

# Main function to handle the image and make predictions
def main(image_path):
    try:
        # Preprocess for ethnicity and emotion
        image_ethnicity_emotion = preprocess_image(image_path, (48, 48))
        nationality, emotion = predict_nationality_and_emotion(image_ethnicity_emotion)
        
        predictions = {'nationality': nationality, 'emotion': emotion}
        
        # Preprocess for age and dress color if required
        if nationality == 'Indian' or nationality == 'White':
            image_age = preprocess_image(image_path, (48, 48))
            age_start, age_end = predict_age(image_age)
            if 10 <= age_start <= 60:
                predictions['age'] = f'{age_start}-{age_end}'
                if nationality == 'Indian':
                    image_color = preprocess_image(image_path, (224, 224), grayscale=False)
                    dress_color = predict_dress_color(image_color)
                    predictions['dress_color'] = dress_color
            else:
                predictions['age'] = "Age not within the specified range."
        
        elif nationality == 'Black':
            image_color = preprocess_image(image_path, (224, 224), grayscale=False)
            dress_color = predict_dress_color(image_color)
            predictions['dress_color'] = dress_color
        
        return predictions

    except ValueError as e:
        return str(e)

# GUI functions
def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        result = main(file_path)
        display_result(result)

def display_result(result):
    result_text = ""
    for key, value in result.items():
        result_text += f"{key}: {value}\n"
    messagebox.showinfo("Prediction Result", result_text)

# Create the main window
root = tk.Tk()
root.title("Image Prediction")

# Create and place the button to open an image
open_button = tk.Button(root, text="Open Image", command=open_image)
open_button.pack(pady=20)

# Run the Tkinter event loop
root.mainloop()





1/1 [==============================] - 0s 21ms/step
